In [1]:
clean_up=True # removes gams-related files in work-folder if true
%run StdPackages.ipynb
os.chdir(directory['py'])
import ShockFunction
import abatement_EOP as abatement
os.chdir(directory['curr'])

The file_gams_py_gdb0.gdx is still active and was not deleted.
The file_gams_py_gdb1.gdx is still active and was not deleted.
The file_gams_py_gdb2.gdx is still active and was not deleted.
The file_gams_py_gdb3.gdx is still active and was not deleted.


In [2]:
def organize_sol_data(sols, x_var, nodes_to_save, node_to_shock, tech_catalog):
    list_of_dfs = []
    for var in sols:
        if var == x_var:
            continue
        df = sols[var]
        index_name = df.index.names[1]
        if len(df.index.get_level_values(index_name).unique()) == 1:
            only_one_series = True
        else:
            only_one_series = False
        df = df.reset_index()
        df2 = df.pivot_table(df.columns[-1], df.columns[0], df.columns[1]).reset_index()
        if tech_catalog in nodes_to_save and not only_one_series:
            df2 = df2[[x_var + "_loop"] + [c for c in nodes_to_save[tech_catalog] if c in df2.columns]]
        df3 = df2.merge(sols[x_var].iloc[sols[x_var].index.get_level_values(1) == node_to_shock].reset_index().drop(columns=sols[x_var].index.names[1]), \
                        how="inner", on=x_var+"_loop").drop(columns=x_var+"_loop")
        df3 = df3.rename(columns={"sol_"+x_var:x_var}).set_index(x_var).sort_values(by=x_var)
        if only_one_series:
            df3.rename(columns={df3.columns[0]:var}, inplace=True)
        else:
            df3.rename(columns={key:var + "_"+ key for key in df3.columns}, inplace=True)
        list_of_dfs.append(df3)
    return list_of_dfs

In [3]:
out = {}

#### Load and setup

In [4]:
#Load model
catalog = "main_catalog"
gm = abatement.abate(pickle_path=directory["gams"] + "/" + catalog, work_folder=directory['work'])
gm.reset_settings()


In [5]:
name = "CO2tax"
x_var = "pM"
node_to_shock = "CO2"

In [6]:
db_end_of_loop = DataBase.GPM_database()

In [7]:
db_end_of_loop[x_var] = gm.get(x_var).copy()
db_end_of_loop[x_var].vals[node_to_shock] = db_end_of_loop[x_var].vals[node_to_shock]*25

In [8]:
(shock_db, shock_kwargs) = ShockFunction.sneaky_db(gm.model.database, db_end_of_loop, n_steps = 150, loop_name=x_var + '_loop')

In [9]:
store_sol = {"M0":{'domains': shock_kwargs['loop_name']}, "PbT":{'domains': shock_kwargs['loop_name']},
             "pM":{'domains': shock_kwargs['loop_name']}, "M":{'domains': shock_kwargs['loop_name']},
             "PwThat":{'domains': shock_kwargs['loop_name']}, "qD":{'domains': shock_kwargs['loop_name']}}

In [10]:
gm.write_and_run(name=name, add_checkpoint=name)

In [11]:
mi = gm.model_instances[name]
shock = mi.std_UEVAS_from_db(shock_db, loop_name=shock_kwargs['loop_name'], shock_name=shock_kwargs['shock_name'], store_sol=store_sol)

In [12]:
mi.execute_shock_from_cp(shock=shock, cp=gm.checkpoints[name])

In [13]:
assert mi.out_db.get(mi.settings.name + "_" + gm.state + "_" + "modelstat")

In [14]:
sols = {key:mi.out_db.series["sol_" + key].vals for key in store_sol.keys()}
nodes_to_save = {
    "techdata_dors_2":["U_ID_t2_1", "U_EOP_t1_1", "U_EOP_t2_1", "U_ID_t1_1", "C_EL_1", "U0_ID_C_EL_1", "CO2", "basetech_EL", "basetech_EL_oil", "oil"]
}

In [15]:
gm.model.database.merge_dbs(gm.model.database,mi.out_db,'second')

In [16]:
out[name] = organize_sol_data(sols, x_var, nodes_to_save, node_to_shock, gm.model.settings.catalog)

In [17]:
df_out = pd.concat(out[name], axis = 1)
df_out.to_pickle(x_var + "_" + gm.model.settings.catalog + ".pkl")

In [18]:
gm.get("map_ID_CU")

MultiIndex([( 'U0_ID_C0_EH',  'C0_EH'),
            ('U0_ID_C_EH_1', 'C_EH_1'),
            ('U0_ID_C_EH_2', 'C_EH_2'),
            (   'U_ID_t1_1', 'C_EH_1'),
            (   'U_ID_t2_1', 'C_EH_2')],
           names=['n', 'nn'])